In [1]:
using HDF5, JLD      # for loading training data
using Plots          # for visualization
using Distributions  # for using all kinds of distributions
using Turing         # yeah - it's Turing.jl

[Turing]: AD chunk size is set as 60


In [2]:
hmm_data = load("hmm-data.jld")["data"]     # load data

plot(1:hmm_data["N"], hmm_data["obs"],      # visualize data
     line=:scatter, lab="obs", m=:circle)
plot!(1:hmm_data["N"], hmm_data["hid_ground"],      # visualize data
      line=:scatter, lab="hid (ground ture)", m=:diamond)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 obs 
 
 
 
 
 hid (ground ture)

In [3]:
# Define a HMM model with data as
#          K := number of hidden stats
#          N := number of observations
#   T_ground := transition matrix
# sig_ground := noise level
#        obs := observations
@model hmm_model(K, N, T_ground, sig_ground, obs) = begin
  hid = tzeros(Int, N)
  hid[1] ~ Categorical(ones(Float64, K) / K)
  obs[1] ~ Normal(hid[1], sqrt(sig_ground))
  for t in 2:N
    hid[t] ~ Categorical(T_ground[:, hid[t - 1]])
    obs[t] ~ Normal(hid[t], sqrt(sig_ground))
  end
end

hmm_model (generic function with 6 methods)

In [ ]:
N_samples = 100 # number of samples to generate

hmm_chn = sample(hmm_model(data=hmm_data), # sample from "hmm_model" with data as "hmm_data" using
                 PG(10, 100))              # "PG" with "10" particles for "100" iterations

[Turing]:  Assume - `hid` is a parameter
  in @~(::Any, ::Any) at compiler.jl:76
[Turing]:  Observe - `obs` is an observation
  in @~(::Any, ::Any) at compiler.jl:57


In [ ]:
plot!(1:hmm_data["N"], hmm_chn[:hid][N_samples],
      line=:scatter, lab="hid (HMM)", m=:star4)

In [ ]:
# Define a HMM model with data as
#       K := number of hidden stats
#       N := number of observations
#       a := param for noise prior (Inv-Gamma)
#       b := param for noise prior (Inv-Gamma)
#   alpha := param for transition matrix prior (Dirichlet)
#     obs := observations
@model bayes_hmm_model(K, N, a, b, alpha, obs) = begin
  # Inverse gamma prior on noise level
  sig ~ InverseGamma(a, b)

  # Dirichlet prior on transition matrix
  T = Vector{Vector{Real}}(K)
  for k = 1:K
    T[k] ~ Dirichlet(alpha)
  end

  hid = tzeros(Int, N)
  hid[1] ~ Categorical(ones(Float64, K) / K)
  obs[1] ~ Normal(hid[1], sqrt(sig))
  for t in 2:N
    hid[t] ~ Categorical(T[hid[t - 1]])
    obs[t] ~ Normal(hid[t], sqrt(sig))
  end
end

In [ ]:
N_samples = 100 # number of samples to generate

bayes_hmm_chn = sample(bayes_hmm_model(data=hmm_data),  # sample from "bayes_hmm_model" with data as "hmm_data" using
                       Gibbs(N_samples,                 # "Gibbs" sampler for "N_samples" iterations by combing
                             PG(10, 1, :hid),           # "PG" with "10" particles for "hid", and
                             HMC(1, 0.2, 3, :T, :sig))) # "HMC" with leapfrog params "0.2" and "3" for "T" and "sig"

In [ ]:
plot!(1:hmm_data["N"], bayes_hmm_chn[:hid][N_samples],
      line=:scatter, lab="hid (BayesHMM)", m=:star5)